## Processing NCI Weatherbench data

In this tutorial, 
- we will use the DLWP data module to fetch and pre-process data from ERA5 to use in a DLWP weather prediction model. 
- For the sake of simplicity, we use only a select few variables over a few years.


In [1]:
from dask.distributed import Client
from datetime import datetime
client = Client(n_workers=12, threads_per_worker=1)  
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 12
Total threads: 12,Total memory: 95.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37045,Workers: 12
Dashboard: /proxy/8787/status,Total threads: 12
Started: Just now,Total memory: 95.00 GiB
Comm: tcp://127.0.0.1:40459,Total threads: 1
Dashboard: /proxy/38213/status,Memory: 7.92 GiB
Nanny: tcp://127.0.0.1:33019,


2024-05-06 10:10:29,957 - distributed.nanny - WARNING - Restarting worker
2024-05-06 10:10:29,960 - distributed.nanny - WARNING - Restarting worker
2024-05-06 10:10:30,125 - distributed.nanny - WARNING - Restarting worker
2024-05-06 10:10:30,131 - distributed.nanny - WARNING - Restarting worker
2024-05-06 10:10:30,430 - distributed.nanny - WARNING - Restarting worker
2024-05-06 10:10:30,436 - distributed.nanny - WARNING - Restarting worker
2024-05-06 10:10:31,446 - distributed.nanny - WARNING - Restarting worker
2024-05-06 10:10:31,451 - distributed.nanny - WARNING - Restarting worker


### Retrieve data

Define the variables and levels we want to retrieve. Single-level variables ignore the "levels" parameter. Also note that not all variables in the ERA5 dataset are coded with their parameter names as of now. We also take a reduced sample of years in the dataset.

In [2]:
variables = ['geopotential', '2m_temperature']
levels = [500]
years = list(range(2013, 2019))

Initialize the data retriever. You'll want to change the directory to where you want to save the files.

In [3]:
! mkdir -p /scratch/vp91/$USER/NCI-DLWP-CS

import os
os.chdir (f"/scratch/vp91/{os.environ['USER']}/NCI-DLWP-CS")
from DLWP.data import ERA5Reanalysis
data_directory = f"/scratch/vp91/{os.environ['USER']}/NCI-DLWP-CS/Data/NCI_tutorial"
os.makedirs(data_directory, exist_ok=True)
era = ERA5Reanalysis(root_directory=data_directory, file_id='NCI_tutorial')
era.set_variables(variables)
era.set_levels(levels)

/opt/conda/envs/nci-dlwp-cs/bin/DLWP/data/era5.py:23: UserWarning: module 'cdsapi' not found; retrieval of ERA5 data unavailable.
  warnings.warn("module 'cdsapi' not found; retrieval of ERA5 data unavailable.")


Download data! Automatically uses multi-processing to retrieve multiple files at a time. Note the parameter `hourly` says we're retrieving only every 3rd hour in the data, which is available hourly. The optional parameter passed to the retrieval package specifies that we want data interpolated to a 2-by-2 latitude-longitude grid.

In [4]:
import glob
import xarray as xr
res = '5.625deg'
print (variables)
all_files = []
DATAPATH = f'/g/data/wb00/NCI-Weatherbench/{res}'
for year in years:
    print(year, end=" ")
    all_files += [ f'{DATAPATH}/{variables[0]}/{variables[0]}_{year}_{res}.nc' ]
    all_files += [ f'{DATAPATH}/{variables[1]}/{variables[1]}_{year}_{res}.nc' ]
    
Dataset = xr.open_mfdataset(all_files, chunks={'time': 10}, parallel = True, join='override') 
#print(Dataset)  
Dataset = Dataset.isel(time=slice(0, None, 3))
era.Dataset = Dataset
era.dataset_dates = era.Dataset['time']
#print(era.Dataset)

['geopotential', '2m_temperature']
2013 2014 2015 2016 2017 2018 

Check that we got what we wanted after the retrieval is done:

In [5]:
print(era.Dataset)

<xarray.Dataset> Size: 2GB
Dimensions:  (time: 17528, lat: 32, lon: 64, level: 13)
Coordinates:
  * time     (time) datetime64[ns] 140kB 2013-01-01 ... 2018-12-31T21:00:00
  * lat      (lat) float64 256B -87.19 -81.56 -75.94 ... 75.94 81.56 87.19
  * lon      (lon) float64 512B 0.0 5.625 11.25 16.88 ... 343.1 348.8 354.4
  * level    (level) int32 52B 50 100 150 200 250 300 ... 600 700 850 925 1000
Data variables:
    t2m      (time, lat, lon) float32 144MB dask.array<chunksize=(4, 32, 64), meta=np.ndarray>
    z        (time, level, lat, lon) float32 2GB dask.array<chunksize=(4, 13, 32, 64), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear


### Process data for ingestion into DLWP

Now we use the DLWP.model.Preprocessor tool to generate a new data file ready for use in a DLWP Keras model. 

Some preliminaries.
- Note that we assign level "0" to the single-level 2m temperature data. It is highly recommend using "pairwise" data processing, which means that each variable is matched to a level pair-wise. 
- The length of the variables and levels lists should be the same. 
- Also note that you only need to specify whole days in the dates. It takes care of the hourly data automatically.

In [6]:
import pandas as pd
from DLWP.data.era5 import get_short_name

dates = list(pd.date_range('2013-01-01', '2018-12-31', freq='D').to_pydatetime())
variables = get_short_name(variables)
levels = [500, 0]
processed_file = '%s/NCI_tutorial_z500_t2m.nc' % data_directory
print ('processed_file:', processed_file )

processed_file: /scratch/vp91/mah900/NCI-DLWP-CS/Data/NCI_tutorial/NCI_tutorial_z500_t2m.nc


Process data.
- For proper use of data in a neural network, variables must be normalized relative to each other. This is typically done simply by removing mean and dividing by standard deviation (`scale_variables` option). 
- To save on memory use, we normally calculate the global mean and std of the data in batches. Since this is a small dataset, we can use a large batch size to make it go faster.

In [7]:
from DLWP.model import Preprocessor

pp = Preprocessor(era, predictor_file=processed_file)
pp.data_to_series(batch_samples=10000, variables=variables, levels=levels, pairwise=True,
                  scale_variables=True, overwrite=True, verbose=True)

2024-05-06 09:53:42.152328: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-06 09:53:44.456414: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-06 09:53:46.393569: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Preprocessor.data_to_samples: opening and formatting raw data
Preprocessor.data_to_samples: creating output file /scratch/vp91/mah900/NCI-DLWP-CS/Data/NCI_tutorial/NCI_tutorial_z500_t2m.nc
Preprocessor.data_to_samples: variable/level pair 1 of 2 (z/500)
Preprocessor.data_to_samples: calculating mean and std


/opt/conda/envs/nci-dlwp-cs/bin/DLWP/model/preprocessing.py:526: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  lat_dim = 'lat' if 'lat' in ds.dims.keys() else 'latitude'
/opt/conda/envs/nci-dlwp-cs/bin/DLWP/model/preprocessing.py:527: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  lon_dim = 'lon' if 'lon' in ds.dims.keys() else 'longitude'
/opt/conda/envs/nci-dlwp-cs/bin/DLWP/model/preprocessing.py:529: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from 

Preprocessor.data_to_samples: writing batch 1 of 2
Preprocessor.data_to_samples: writing batch 2 of 2
Preprocessor.data_to_samples: variable/level pair 2 of 2 (t2m/0)
Preprocessor.data_to_samples: calculating mean and std
Preprocessor.data_to_samples: writing batch 1 of 2
Preprocessor.data_to_samples: writing batch 2 of 2


Show our dataset, then clean up. We also save to a version with no string coordinates (might be needed for tempest-remap in the next tutorial).

In [8]:
print(pp.data)
pp.data.drop('varlev').to_netcdf(processed_file + '.nocoord')
era.close()
pp.close()

<xarray.Dataset> Size: 287MB
Dimensions:     (lat: 32, lon: 64, varlev: 2, sample: 17528)
Coordinates:
  * lat         (lat) float32 128B -87.19 -81.56 -75.94 ... 75.94 81.56 87.19
  * lon         (lon) float32 256B 0.0 5.625 11.25 16.88 ... 343.1 348.8 354.4
  * varlev      (varlev) <U5 40B 'z/500' 't2m/0'
  * sample      (sample) datetime64[ns] 140kB 2013-01-01 ... 2018-12-31T21:00:00
Data variables:
    predictors  (sample, varlev, lat, lon) float32 287MB dask.array<chunksize=(1, 2, 32, 64), meta=np.ndarray>
    mean        (varlev) float32 8B dask.array<chunksize=(2,), meta=np.ndarray>
    std         (varlev) float32 8B dask.array<chunksize=(2,), meta=np.ndarray>
Attributes:
    description:  Training data for DLWP
    scaling:      True


/jobfs/115046880.gadi-pbs/ipykernel_514936/2584461264.py:2: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  pp.data.drop('varlev').to_netcdf(processed_file + '.nocoord')
